Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
import pandas as pd
df = pd.read_csv('ratings.csv')
def class_films(rating):
    if rating <= 2: 
        rating = 'низкий рейтинг'
    elif rating <= 4:
        rating = 'средний рейтинг'
    elif rating <= 5:
        rating = 'высокий рейтинг'
    return rating       
df['class'] = df['rating'].apply(class_films)
df.head(10)

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг
5,1,1263,2.0,1260759151,низкий рейтинг
6,1,1287,2.0,1260759187,низкий рейтинг
7,1,1293,2.0,1260759148,низкий рейтинг
8,1,1339,3.5,1260759125,средний рейтинг
9,1,1343,2.0,1260759131,низкий рейтинг


Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. 
Т.е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = ....
Результат классификации запишите в отдельный столбец region.

In [280]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
df = pd.read_csv('keywords.csv')
def geo_class (keyword):
    '''
    функция ищет в поисковом запросе названия городов ищз словаря geo_data. Если находит,
    то проставляет название регионаб если не находит то слово undifined.
    '''
    region = 'undifined'
    #огранизуем цикл, что бы достать элементы списка(города), входящие в словарь
    for key,value in geo_data.items():
        for items in value:
            #если город из нашего словаря geo_data есть в поле kewords, то region = названию региона(ключ словаря)
            if items in keyword:
                region = key 
    return region
# с помощью метода apply применяем формулу geo_class ко всему столбцу 'keywords'
df['region'] = df['keyword'].apply(geo_class)
df['region'].value_counts()

undifined         99253
Центр               385
Северо-Запад        268
Дальний Восток       94
Name: region, dtype: int64

Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга



In [6]:
df_ratings = pd.read_csv('ratings.csv')
#считаем средний рейтинг с группировкой по столбцу movieid
mean_df_ratings = df_ratings.groupby('movieId').mean()
df_movies = pd.read_csv('movies.csv')
#создаем список с годами для нахождения года в описании фильма и вынесении полученного года в отдельный столбец year
years = range(1950,2011)
def production_year(title):
    '''
    для поиска года выпуска фильма- ищем вхождения списка с годами() в контекст строки title
    '''
    year = 1900
    for item in years:
        i= str(item)
        if i in title:
            year = item
    return year
df_movies['year'] =  df_movies['title'].apply(production_year)
#обогощаем датафрейм с рейтингм данными с датафрейма с фильиами (ключевое поле movieId)
df_merged = pd.merge(mean_df_ratings,df_movies,left_index=True,right_on='movieId' )
#оставляем для отчета только 2 столбца,группируем по годам, вычисляея средний рейтинг, сортируем по убыванию столбец "Рейтинг"
df_merged[['rating','year']].groupby('year').mean().sort_values('rating', ascending=False).head(20)

,rating
year,
1951,3.906653
1952,3.802318
1962,3.772962
1953,3.738878
1957,3.693322
1973,3.688212
1971,3.643227
1969,3.604509
1972,3.585295
